# Creazione Training Dataset

## Inizializzazione

In [3]:
!pip install accelerate -U > /dev/null 2>&1
!pip install datasets > /dev/null 2>&1
!pip install --upgrade transformers datasets accelerate > /dev/null 2>&1
!pip install torch > /dev/null 2>&1

In [4]:
import nltk

# Scarica il dataset wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from datasets import load_dataset

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

# Numero di righe da stampare
num_rows_to_print = 1

# Stampa le prime 10 righe della colonna 'wsd' per ciascuna suddivisione
# for split in ['train', 'validation', 'test']:
#     print(f"Prime {num_rows_to_print} righe della colonna 'wsd' per la suddivisione {split}:")
#     wsd_column = ds[split]['wsd']
#     for i in range(min(num_rows_to_print, len(wsd_column))):
#         print(wsd_column[i])
#     print("\n")  # Stampa una riga vuota tra le suddivisioni

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/51086 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2288 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2287 [00:00<?, ? examples/s]

## WSD

### Cambio Aggettivi Sinonimi Premessa

Dimensione del dataset modificato: 29286


In [6]:
import nltk
from nltk.corpus import wordnet as wn
import random
from datasets import Dataset, DatasetDict
from tqdm import tqdm  # Per la barra di progresso

# Supponiamo che 'ds' sia già definito e contenga il dataset.
# Funzione per ottenere i sinonimi
def get_synonyms(word):
    """Restituisce i sinonimi di una parola, escludendo la parola stessa."""
    synonyms = set()
    for synset in wn.synsets(word, pos=wn.ADJ):
        for lemma in synset.lemmas():
            synonym = lemma.name()
            if synonym != word:
                synonyms.add(synonym)
    return list(synonyms)

# Funzione per sostituire un aggettivo con un sinonimo
def replace_random_adjective(sentence, adjectives):
    """Sostituisce casualmente uno degli aggettivi nella frase con un sinonimo,
       escludendo le parole che iniziano con una lettera maiuscola."""
    if not adjectives:
        return sentence, sentence  # Se non ci sono aggettivi, restituisce la stessa frase

    # Filtra aggettivi che non iniziano con maiuscola
    lowercase_adjectives = [adj for adj in adjectives if not adj[0].isupper()]

    if not lowercase_adjectives:
        return sentence, sentence  # Se non ci sono aggettivi modificabili, restituisce la stessa frase

    # Seleziona un aggettivo casuale
    adjective_to_replace = random.choice(lowercase_adjectives)
    synonyms = get_synonyms(adjective_to_replace)

    if not synonyms:
        return sentence, sentence  # Se non ci sono sinonimi, restituisce la stessa frase

    # Seleziona un sinonimo casuale
    replacement = random.choice(synonyms)

    # Se l'aggettivo da sostituire inizia con una lettera maiuscola, non sostituirlo
    if adjective_to_replace[0].isupper():
        return sentence, sentence  # Nessuna sostituzione

    # Sostituisci l'aggettivo nella frase con il sinonimo
    new_sentence = sentence.replace(adjective_to_replace, replacement, 1)

    return sentence, new_sentence

# Funzione per processare una frase
def process_sentence(wsd_info, sentence_key):
    """Elabora una frase per sostituire casualmente un aggettivo con un sinonimo."""
    # Costruisci la frase a partire dai dati
    sentence = " ".join(word_info['text'] for word_info in wsd_info[sentence_key])

    # Raccolta degli aggettivi
    adjectives = [word_info['text'] for word_info in wsd_info[sentence_key] if word_info['pos'] == 'ADJ']

    # Sostituzione e restituzione
    original_sentence, modified_sentence = replace_random_adjective(sentence, adjectives)

    return original_sentence, modified_sentence

# Elenco per le frasi modificate
modified_samples = []

# Ottieni il numero totale di campioni
total_samples = len(ds['train'])
max_samples = 7500  # Limite massimo di campioni

# Processa tutte le frasi nel dataset con una barra di progresso
for i in tqdm(range(total_samples), desc="Elaborazione del Dataset"):
    if len(modified_samples) >= max_samples:
        break  # Fermati se il limite è stato raggiunto

    sample = ds['train'][i]
    wsd_info = sample['wsd']

    # Processa la premessa
    original_premise, modified_premise = process_sentence(wsd_info, 'premise')
    original_hypothesis = " ".join(word_info['text'] for word_info in wsd_info['hypothesis'])

    # Aggiungi al dizionario solo se la premessa è stata modificata
    if original_premise != modified_premise:
        modified_samples.append({
            'id': sample['id'],
            'premise': modified_premise,
            'hypothesis': original_hypothesis,
            'label': sample['label'],
            'wsd': wsd_info,
            'srl': sample['srl']
        })

# Crea il dizionario finale
modified_dataset_adjectives = DatasetDict({
    'train': Dataset.from_list(modified_samples)
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset_adjectives['train'])}")

Elaborazione del Dataset:  25%|██▌       | 13015/51086 [00:52<02:34, 246.23it/s]


Dimensione del dataset modificato: 7500


In [7]:
# Estrai la premessa, l'ipotesi e l'etichetta dal dataset originale per il campione specificato
original_sample = ds['train'][0]
original_premise = original_sample['premise']
original_hypothesis = original_sample['hypothesis']
original_label = original_sample['label']

# Estrai la premessa, l'ipotesi e l'etichetta dal dataset modificato per il campione specificato
modified_sample = modified_dataset_adjectives['train'][0]
modified_premise = modified_sample['premise']
modified_hypothesis = modified_sample['hypothesis']
modified_label = modified_sample['label']

# Stampa le premesse, le ipotesi e le etichette
print("Premessa del campione originale:")
print(original_premise)

print("\nPremessa del campione modificato:")
print(modified_premise)

print("\nIpotesi del campione originale:")
print(original_hypothesis)

print("\nIpotesi del campione modificato:")
print(modified_hypothesis)

print("\nLabel del campione originale:")
print(original_label)

print("\nLabel del campione modificato:")
print(modified_label)


Premessa del campione originale:
Roman Atwood . He is best known for his vlogs , where he posts updates about his life on a daily basis . His vlogging channel , `` RomanAtwoodVlogs '' , has a total of 3.3 billion views and 11.9 million subscribers . He also has another YouTube channel called `` RomanAtwood '' , where he posts pranks .

Premessa del campione modificato:
Roman Atwood . He is best known for his vlogs , where he posts updates about his life on a day-by-day basis . His vlogging channel , ` ` RomanAtwoodVlogs '' , has a total of 3.3 billion views and 11.9 million subscribers . He also has another YouTube channel called ` ` RomanAtwood '' , where he posts pranks .

Ipotesi del campione originale:
Roman Atwood is a content creator.

Ipotesi del campione modificato:
Roman Atwood is a content creator .

Label del campione originale:
ENTAILMENT

Label del campione modificato:
ENTAILMENT


### Cambio Aggettivi Sinonimi Ipotesi

Dimensione del dataset modificato: 7678



In [8]:
import nltk
from nltk.corpus import wordnet as wn
import random
from datasets import Dataset, DatasetDict
from tqdm import tqdm  # Per la barra di progresso

# Supponiamo che 'ds' sia già definito e contenga il dataset.
# Funzione per ottenere i sinonimi
def get_synonyms(word):
    """Restituisce i sinonimi di una parola, escludendo la parola stessa e scegliendo il migliore."""
    synonyms = {}
    for synset in wn.synsets(word, pos=wn.ADJ):
        for lemma in synset.lemmas():
            synonym = lemma.name()
            if synonym != word:
                # Aggiungi il sinonimo al dizionario con la frequenza di uso come valore
                if synonym not in synonyms or lemma.count() > synonyms[synonym].count():
                    synonyms[synonym] = lemma
    return list(synonyms.keys())

# Funzione per sostituire un aggettivo con un sinonimo
def replace_best_adjective(sentence, adjectives):
    """Sostituisce l'aggettivo con il miglior sinonimo disponibile, escludendo le parole che iniziano con una lettera maiuscola."""
    if not adjectives:
        return sentence, sentence  # Se non ci sono aggettivi, restituisce la stessa frase

    # Filtra aggettivi che non iniziano con maiuscola
    lowercase_adjectives = [adj for adj in adjectives if not adj[0].isupper()]

    if not lowercase_adjectives:
        return sentence, sentence  # Se non ci sono aggettivi modificabili, restituisce la stessa frase

    # Seleziona un aggettivo casuale
    adjective_to_replace = random.choice(lowercase_adjectives)
    synonyms = get_synonyms(adjective_to_replace)

    if not synonyms:
        return sentence, sentence  # Se non ci sono sinonimi, restituisce la stessa frase

    # Seleziona il miglior sinonimo disponibile
    replacement = synonyms[0]  # Prendi il primo sinonimo nella lista (già ordinata)

    # Se l'aggettivo da sostituire inizia con una lettera maiuscola, non sostituirlo
    if adjective_to_replace[0].isupper():
        return sentence, sentence  # Nessuna sostituzione

    # Sostituisci l'aggettivo nella frase con il sinonimo
    new_sentence = sentence.replace(adjective_to_replace, replacement, 1)

    return sentence, new_sentence

# Funzione per processare una frase
def process_sentence(wsd_info, sentence_key):
    """Elabora una frase per sostituire un aggettivo con un sinonimo."""
    # Costruisci la frase a partire dai dati
    sentence = " ".join(word_info['text'] for word_info in wsd_info[sentence_key])

    # Raccolta degli aggettivi
    adjectives = [word_info['text'] for word_info in wsd_info[sentence_key] if word_info['pos'] == 'ADJ']

    # Sostituzione e restituzione
    original_sentence, modified_sentence = replace_best_adjective(sentence, adjectives)

    return original_sentence, modified_sentence

# Elenco per le frasi modificate
modified_samples = []

# Ottieni il numero totale di campioni
total_samples = len(ds['train'])

# Processa tutte le frasi nel dataset con una barra di progresso
for i in tqdm(range(total_samples), desc="Elaborazione del Dataset"):
    sample = ds['train'][i]
    wsd_info = sample['wsd']

    # Processa l'ipotesi
    original_hypothesis, modified_hypothesis = process_sentence(wsd_info, 'hypothesis')
    original_premise = " ".join(word_info['text'] for word_info in wsd_info['premise'])

    # Aggiungi al dizionario solo se l'ipotesi è stata modificata
    if original_hypothesis != modified_hypothesis:
        modified_samples.append({
            'id': sample['id'],
            'premise': original_premise,
            'hypothesis': modified_hypothesis,
            'label': sample['label'],
            'wsd': wsd_info,
            'srl': sample['srl']
        })

# Crea il dizionario finale
modified_dataset_adjectives_2 = DatasetDict({
    'train': Dataset.from_list(modified_samples)
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset_adjectives_2['train'])}")

Elaborazione del Dataset: 100%|██████████| 51086/51086 [02:35<00:00, 329.18it/s]


Dimensione del dataset modificato: 7707


In [9]:
# Estrai la premessa, l'ipotesi e l'etichetta dal dataset originale per il campione specificato
original_sample = ds['train'][14]
original_premise = original_sample['premise']
original_hypothesis = original_sample['hypothesis']
original_label = original_sample['label']

# Estrai la premessa, l'ipotesi e l'etichetta dal dataset modificato per il campione specificato
modified_sample = modified_dataset_adjectives_2['train'][1]
modified_premise = modified_sample['premise']
modified_hypothesis = modified_sample['hypothesis']
modified_label = modified_sample['label']

# Stampa le premesse, le ipotesi e le etichette
print("Premessa del campione originale:")
print(original_premise)

print("\nPremessa del campione modificato:")
print(modified_premise)

print("\nIpotesi del campione originale:")
print(original_hypothesis)

print("\nIpotesi del campione modificato:")
print(modified_hypothesis)

print("\nLabel del campione originale:")
print(original_label)

print("\nLabel del campione modificato:")
print(modified_label)


Premessa del campione originale:
Lisbon ( [ ˈlɪzbən ] Lisboa , [ liʒˈβoɐ ] ) is the capital and the largest city of Portugal , with a population of 552,700 within its administrative limits in an area of 100.05 km ² .

Premessa del campione modificato:
Lisbon ( [ ˈlɪzbən ] Lisboa , [ liʒˈβoɐ ] ) is the capital and the largest city of Portugal , with a population of 552,700 within its administrative limits in an area of 100.05 km ² .

Ipotesi del campione originale:
Lisbon has a population larger than 1.

Ipotesi del campione modificato:
Lisbon has a population bigger than 1 .

Label del campione originale:
ENTAILMENT

Label del campione modificato:
ENTAILMENT


### Cambio aggettivi Antonimi Ipotesi

Dimensione del dataset modificato: 7142



In [10]:
import nltk
from nltk.corpus import wordnet as wn
from datasets import Dataset, DatasetDict
from tqdm import tqdm  # Importa tqdm per la barra di progresso

# Supponiamo che 'ds' sia già definito e contenga il dataset.
# Estrai l'intero dataset
dataset = ds['train']

def get_adjectives_and_antonyms(sentence):
    """Restituisce aggettivi e i loro antonimi (se presenti), escludendo quelli con lettere maiuscole."""
    adjectives = []
    antonyms = {}

    # Estrai le parole e i loro POS tags dalla frase
    words = [(word_info['text'], word_info['pos']) for word_info in sentence]

    # Trova gli aggettivi e i loro antonimi
    for word, pos in words:
        if pos == 'ADJ':
            # Solo aggettivi con lettere minuscole vengono considerati
            if word[0].islower():
                adjectives.append(word)
                synsets = wn.synsets(word, pos=wn.ADJ)
                antonyms_for_word = {}
                for synset in synsets:
                    for lemma in synset.lemmas():
                        if lemma.antonyms():
                            for ant in lemma.antonyms():
                                antonym = ant.name()
                                # Se l'antonomimo non è ancora stato aggiunto o ha una frequenza di uso maggiore, aggiorna
                                if antonym not in antonyms_for_word or ant.count() > antonyms_for_word[antonym].count():
                                    antonyms_for_word[antonym] = ant
                if antonyms_for_word:
                    antonyms[word] = list(antonyms_for_word.keys())

    return adjectives, antonyms

def replace_adjectives_with_antonyms(sentence, antonyms):
    """Sostituisce gli aggettivi con i loro antonimi nella frase."""
    words = sentence.split()
    modified_words = []

    for word in words:
        # Sostituisci l'aggettivo con il suo antonimo se presente
        if word in antonyms:
            # Usa il primo antonimo disponibile
            antonym = antonyms[word][0]
            modified_words.append(antonym)
        else:
            modified_words.append(word)

    return ' '.join(modified_words)

def modify_label(label):
    """Modifica la label secondo le regole specificate."""
    if label == 'ENTAILMENT':
        return 'CONTRADICTION'
    elif label == 'CONTRADICTION':
        return 'ENTAILMENT'
    else:
        return label

# Lista per memorizzare i campioni modificati
modified_samples = []

# Barra di progresso
for sample in tqdm(dataset, desc="Elaborazione del dataset", unit="campione"):
    # Estrai le informazioni WSD
    wsd_info = sample['wsd']

    # Costruisci la frase a partire dai dati
    hypothesis = " ".join(word_info['text'] for word_info in wsd_info['hypothesis'])

    # Trova aggettivi e antonimi
    adjectives, antonyms = get_adjectives_and_antonyms(wsd_info['hypothesis'])

    # Modifica la frase
    modified_hypothesis = replace_adjectives_with_antonyms(hypothesis, antonyms)

    # Modifica la label
    new_label = modify_label(sample['label'])

    # Aggiungi al dizionario solo se la frase è stata modificata
    if hypothesis != modified_hypothesis:
        modified_samples.append({
            'id': sample['id'],
            'premise': sample['premise'],
            'hypothesis': modified_hypothesis,
            'label': new_label,
            'wsd': wsd_info,
            'srl': sample['srl']
        })

# Crea il dizionario finale
modified_dataset_antonym = DatasetDict({
    'train': Dataset.from_dict({
        'id': [s['id'] for s in modified_samples],
        'premise': [s['premise'] for s in modified_samples],
        'hypothesis': [s['hypothesis'] for s in modified_samples],
        'label': [s['label'] for s in modified_samples],
        'wsd': [s['wsd'] for s in modified_samples],
        'srl': [s['srl'] for s in modified_samples]
    })
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset_antonym['train'])}")

Elaborazione del dataset: 100%|██████████| 51086/51086 [02:18<00:00, 368.88campione/s]


Dimensione del dataset modificato: 7142


In [11]:
# Estrai la premessa, l'ipotesi e l'etichetta dal dataset originale per il campione specificato
original_sample = ds['train'][14]
original_premise = original_sample['premise']
original_hypothesis = original_sample['hypothesis']
original_label = original_sample['label']

# Estrai la premessa, l'ipotesi e l'etichetta dal dataset modificato per il campione specificato
modified_sample = modified_dataset_antonym['train'][1]
modified_premise = modified_sample['premise']
modified_hypothesis = modified_sample['hypothesis']
modified_label = modified_sample['label']

# Stampa le premesse, le ipotesi e le etichette
print("Premessa del campione originale:")
print(original_premise)

print("\nPremessa del campione modificato:")
print(modified_premise)

print("\nIpotesi del campione originale:")
print(original_hypothesis)

print("\nIpotesi del campione modificato:")
print(modified_hypothesis)

print("\nLabel del campione originale:")
print(original_label)

print("\nLabel del campione modificato:")
print(modified_label)


Premessa del campione originale:
Lisbon ( [ ˈlɪzbən ] Lisboa , [ liʒˈβoɐ ] ) is the capital and the largest city of Portugal , with a population of 552,700 within its administrative limits in an area of 100.05 km ² .

Premessa del campione modificato:
Lisbon ( [ ˈlɪzbən ] Lisboa , [ liʒˈβoɐ ] ) is the capital and the largest city of Portugal , with a population of 552,700 within its administrative limits in an area of 100.05 km ² .

Ipotesi del campione originale:
Lisbon has a population larger than 1.

Ipotesi del campione modificato:
Lisbon has a population small than 1 .

Label del campione originale:
ENTAILMENT

Label del campione modificato:
CONTRADICTION


### Cambio verbo da positivo a negativo

Dimensione del dataset modificato: 7500



In [12]:
# Assicurati di aver scaricato le risorse necessarie
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [13]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import random

# Supponiamo che 'ds' sia già definito e contenga il dataset
dataset = ds['train']

def negate_verb(verb, pos_tag):
    """Negare un verbo mantenendo lo stesso tempo verbale."""
    if pos_tag.startswith('VB'):
        if pos_tag == 'VB':
            return f"do not {verb}"
        elif pos_tag == 'VBD':
            return f"did not {verb}"
        elif pos_tag == 'VBG':
            return f"not {verb}"
        elif pos_tag == 'VBN':
            return f"not {verb}"
        elif pos_tag == 'VBP':
            return f"do not {verb}"
        elif pos_tag == 'VBZ':
            return f"does not {verb}"
    return verb

def modify_label(label):
    """Modifica la label secondo le regole specificate."""
    if label == 'ENTAILMENT':
        return 'CONTRADICTION'
    elif label == 'CONTRADICTION':
        return 'ENTAILMENT'
    else:
        return label

# Lista per memorizzare i campioni modificati
modified_samples = []

# Lista per memorizzare i campioni interessanti
interesting_samples = []

# Barra di progresso
for sample in tqdm(dataset, desc="Elaborazione del Dataset", unit="campione"):
    wsd_info = sample['wsd']

    # Costruisci l'ipotesi a partire dai dati
    hypothesis = " ".join(word_info['text'] for word_info in wsd_info['hypothesis'])

    # Tokenizza la frase e applica il POS tagging
    tokens = word_tokenize(hypothesis)
    tagged = nltk.pos_tag(tokens)

    # Lista per memorizzare i verbi modificati
    modified_words = []
    has_verbs = False

    # Processa i token per negare i verbi
    for word, pos in tagged:
        if pos.startswith('VB'):
            has_verbs = True
            modified_word = negate_verb(word, pos)
            modified_words.append(modified_word)
        else:
            modified_words.append(word)

    # Costruisci la frase modificata
    modified_hypothesis = ' '.join(modified_words)

    # Modifica la label
    new_label = modify_label(sample['label'])

    # Aggiungi al dizionario solo se la frase è stata modificata
    if hypothesis != modified_hypothesis and has_verbs:
        interesting_samples.append({
            'id': sample['id'],
            'premise': sample['premise'],
            'hypothesis': modified_hypothesis,
            'label': new_label,
            'wsd': wsd_info,
            'srl': sample['srl']
        })

# Seleziona un campione casuale di circa 7000-7500 campioni se ci sono abbastanza dati
random.seed(42)  # Per riproducibilità
selected_samples = random.sample(interesting_samples, min(2000, len(interesting_samples)))

# Crea il dizionario finale
modified_dataset_verbs = DatasetDict({
    'train': Dataset.from_dict({
        'id': [s['id'] for s in selected_samples],
        'premise': [s['premise'] for s in selected_samples],
        'hypothesis': [s['hypothesis'] for s in selected_samples],
        'label': [s['label'] for s in selected_samples],
        'wsd': [s['wsd'] for s in selected_samples],
        'srl': [s['srl'] for s in selected_samples]
    })
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset_verbs['train'])}")

Elaborazione del Dataset: 100%|██████████| 51086/51086 [03:41<00:00, 231.15campione/s]


Dimensione del dataset modificato: 2000


In [14]:
# Estrai la premessa, l'ipotesi e l'etichetta dal dataset originale per il campione specificato
original_sample = ds['train'][100]
original_premise = original_sample['premise']
original_hypothesis = original_sample['hypothesis']
original_label = original_sample['label']

# Estrai la premessa, l'ipotesi e l'etichetta dal dataset modificato per il campione specificato
modified_sample = modified_dataset_verbs['train'][100]
modified_premise = modified_sample['premise']
modified_hypothesis = modified_sample['hypothesis']
modified_label = modified_sample['label']

# Stampa le premesse, le ipotesi e le etichette
print("Premessa del campione originale:")
print(original_premise)

print("\nPremessa del campione modificato:")
print(modified_premise)

print("\nIpotesi del campione originale:")
print(original_hypothesis)

print("\nIpotesi del campione modificato:")
print(modified_hypothesis)

print("\nLabel del campione originale:")
print(original_label)

print("\nLabel del campione modificato:")
print(modified_label)


Premessa del campione originale:
In comparative mythology , sky father is a term for a recurring concept of a sky god who is addressed as a `` father '' , often the father of a pantheon . Zeus . He was respected as an allfather who was chief of the gods and assigned the others to their roles : `` Even the gods who are not his natural children address him as Father , and all the gods rise in his presence . ''

Premessa del campione modificato:
Sharon Tate . In January 20 , 1968 , Tate married Roman Polanski , her director and co-star in 1967 's The Fearless Vampire Killers .

Ipotesi del campione originale:
Zeus is an angel.

Ipotesi del campione modificato:
Sharon Tate did not had a husband who did not was an optimist .

Label del campione originale:
CONTRADICTION

Label del campione modificato:
NEUTRAL


### Iperonimo

Dimensione del dataset modificato: 38909


In [15]:
# Assicurati di aver scaricato i pacchetti necessari di nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
import nltk
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from tqdm import tqdm  # Per la barra di progresso
from datasets import Dataset, DatasetDict

# Supponiamo che 'ds' sia già definito e contenga il dataset.
# Elenco per memorizzare i campioni modificati
modified_samples = []

# Lista di articoli da escludere
articles = ['a', 'an', 'the']

# Numero massimo di campioni da modificare
max_samples = 1000

def get_hypernym(word, pos_tag):
    """Restituisce un iperonimo rilevante per una parola data, escludendo i verbi."""
    if pos_tag.startswith('NN'):  # Verifica che sia un sostantivo
        synsets = wn.synsets(word, pos=wn.NOUN)
        if synsets:
            hypernyms = synsets[0].hypernyms()  # Prendi il primo iperonimo disponibile
            if hypernyms:
                return hypernyms[0].lemmas()[0].name()  # Restituisce il primo iperonimo
    return None

def process_sample(sample):
    """Elabora un singolo campione sostituendo una parola con il suo iperonimo."""
    hypothesis = " ".join(word_info['text'] for word_info in sample['wsd']['hypothesis'])

    # Tokenizza la frase e aggiungi POS tagging
    tokens = word_tokenize(hypothesis)
    tagged_tokens = pos_tag(tokens)

    # Trova la prima parola con un iperonimo rilevante e sostituiscila
    modified_tokens = tokens[:]  # Copia dei token originali per modificarli
    for i, (token, pos) in enumerate(tagged_tokens):
        # Salta articoli, parole con maiuscole e verbi
        if token.lower() in articles or token[0].isupper() or pos.startswith('VB'):
            continue
        hypernym = get_hypernym(token, pos)
        if hypernym:
            modified_tokens[i] = hypernym  # Sostituisci la parola con l'iperonimo
            break

    # Se la frase è cambiata, aggiungila al dizionario modificato
    original_sentence = " ".join(tokens)
    modified_sentence = " ".join(modified_tokens)

    if original_sentence != modified_sentence:
        return {
            'id': sample['id'],
            'premise': sample['premise'],
            'hypothesis': modified_sentence,
            'label': sample['label'],
            'wsd': sample['wsd'],
            'srl': sample['srl']
        }
    return None

# Processa tutto il dataset con una barra di progresso
for sample in tqdm(ds['train'], desc="Elaborazione del dataset", unit="campione"):
    if len(modified_samples) >= max_samples:
        break  # Fermati se il limite è stato raggiunto
    modified_sample = process_sample(sample)
    if modified_sample:
        modified_samples.append(modified_sample)

# Crea il dizionario finale
modified_dataset_hypernyms = DatasetDict({
    'train': Dataset.from_dict({
        'id': [s['id'] for s in modified_samples],
        'premise': [s['premise'] for s in modified_samples],
        'hypothesis': [s['hypothesis'] for s in modified_samples],
        'label': [s['label'] for s in modified_samples],
        'wsd': [s['wsd'] for s in modified_samples],
        'srl': [s['srl'] for s in modified_samples]
    })
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset_hypernyms['train'])}")

Elaborazione del dataset:   3%|▎         | 1309/51086 [00:07<04:58, 166.67campione/s]


Dimensione del dataset modificato: 1000


In [17]:
import nltk
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from tqdm import tqdm  # Per la barra di progresso
from datasets import Dataset, DatasetDict

# Supponiamo che 'ds' sia già definito e contenga il dataset.
# Elenco per memorizzare i campioni modificati
modified_samples = []

# Lista di articoli da escludere
articles = ['a', 'an', 'the']

# Numero massimo di campioni da modificare
max_samples = 1000

def get_hypernym(word, pos_tag):
    """Restituisce un iperonimo rilevante per una parola data, escludendo i verbi."""
    if pos_tag.startswith('NN'):  # Verifica che sia un sostantivo
        synsets = wn.synsets(word, pos=wn.NOUN)
        if synsets:
            hypernyms = synsets[0].hypernyms()  # Prendi il primo iperonimo disponibile
            if hypernyms:
                return hypernyms[0].lemmas()[0].name()  # Restituisce il primo iperonimo
    return None

def process_sample(sample):
    """Elabora un singolo campione sostituendo una parola con il suo iperonimo."""
    hypothesis = " ".join(word_info['text'] for word_info in sample['wsd']['hypothesis'])

    # Tokenizza la frase e aggiungi POS tagging
    tokens = word_tokenize(hypothesis)
    tagged_tokens = pos_tag(tokens)

    # Trova la prima parola con un iperonimo rilevante e sostituiscila
    modified_tokens = tokens[:]  # Copia dei token originali per modificarli
    for i, (token, pos) in enumerate(tagged_tokens):
        # Salta articoli, parole con maiuscole e verbi
        if token.lower() in articles or token[0].isupper() or pos.startswith('VB'):
            continue
        hypernym = get_hypernym(token, pos)
        if hypernym:
            modified_tokens[i] = hypernym  # Sostituisci la parola con l'iperonimo
            break

    # Se la frase è cambiata, aggiungila al dizionario modificato
    original_sentence = " ".join(tokens)
    modified_sentence = " ".join(modified_tokens)

    if original_sentence != modified_sentence:
        return {
            'id': sample['id'],
            'premise': sample['premise'],
            'hypothesis': modified_sentence,
            'label': sample['label'],
            'wsd': sample['wsd'],
            'srl': sample['srl'],
            'original_hypothesis': original_sentence  # Aggiunta per la stampa successiva
        }
    return None

# Processa tutto il dataset con una barra di progresso
for idx, sample in enumerate(tqdm(ds['train'], desc="Elaborazione del dataset", unit="campione")):
    if len(modified_samples) >= max_samples:
        break  # Fermati se il limite è stato raggiunto
    modified_sample = process_sample(sample)
    if modified_sample:
        modified_samples.append(modified_sample)

        # Stampa la prima ipotesi originale e modificata, poi fermati
        if idx == 13:
            print(f"Ipotesi originale: {modified_sample['original_hypothesis']}")
            print(f"Ipotesi modificata: {modified_sample['hypothesis']}")
            break  # Ferma l'esecuzione qui

# Crea il dizionario finale
modified_dataset_hypernyms = DatasetDict({
    'train': Dataset.from_dict({
        'id': [s['id'] for s in modified_samples],
        'premise': [s['premise'] for s in modified_samples],
        'hypothesis': [s['hypothesis'] for s in modified_samples],
        'label': [s['label'] for s in modified_samples],
        'wsd': [s['wsd'] for s in modified_samples],
        'srl': [s['srl'] for s in modified_samples]
    })
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset_hypernyms['train'])}")


Elaborazione del dataset:   0%|          | 13/51086 [00:00<03:55, 216.89campione/s]

Ipotesi originale: Grace Jones is a dancer .
Ipotesi modificata: Grace Jones is a performer .


Dimensione del dataset modificato: 11


In [18]:
# Estrai la premessa, l'ipotesi e l'etichetta dal dataset originale per il campione specificato
original_sample = ds['train'][2]
original_hypothesis = original_sample['hypothesis']

# Estrai la premessa, l'ipotesi e l'etichetta dal dataset modificato per il campione specificato
modified_sample = modified_dataset_hypernyms['train'][1]
modified_hypothesis = modified_sample['hypothesis']

# Stampa le premesse, le ipotesi e le etichette

print("\nIpotesi del campione originale:")
print(original_hypothesis)

print("\nIpotesi del campione modificato:")
print(modified_hypothesis)



Ipotesi del campione originale:
There is a movie called The Hunger Games.

Ipotesi del campione modificato:
There is a product called The Hunger Games .


## SRL

### Inversione Agent e Patient
Dimensione del dataset modificato: 2843



In [19]:
import nltk

# Scarica il dataset wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
from datasets import load_dataset

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

# Numero di righe da stampare
num_rows_to_print = 1

# Stampa le prime 10 righe della colonna 'wsd' per ciascuna suddivisione
for split in ['train']:
    print(f"Prime {num_rows_to_print} righe della colonna 'srl' per la suddivisione {split}:")
    wsd_column = ds[split]['srl']
    for i in range(min(num_rows_to_print, len(wsd_column))):
        print(wsd_column[i])
    print("\n")  # Stampa una riga vuota tra le suddivisioni

Prime 1 righe della colonna 'srl' per la suddivisione train:
{'premise': {'tokens': [{'index': 0, 'rawText': 'Roman'}, {'index': 1, 'rawText': 'Atwood'}, {'index': 2, 'rawText': '.'}, {'index': 3, 'rawText': 'He'}, {'index': 4, 'rawText': 'is'}, {'index': 5, 'rawText': 'best'}, {'index': 6, 'rawText': 'known'}, {'index': 7, 'rawText': 'for'}, {'index': 8, 'rawText': 'his'}, {'index': 9, 'rawText': 'vlogs'}, {'index': 10, 'rawText': ','}, {'index': 11, 'rawText': 'where'}, {'index': 12, 'rawText': 'he'}, {'index': 13, 'rawText': 'posts'}, {'index': 14, 'rawText': 'updates'}, {'index': 15, 'rawText': 'about'}, {'index': 16, 'rawText': 'his'}, {'index': 17, 'rawText': 'life'}, {'index': 18, 'rawText': 'on'}, {'index': 19, 'rawText': 'a'}, {'index': 20, 'rawText': 'daily'}, {'index': 21, 'rawText': 'basis'}, {'index': 22, 'rawText': '.'}, {'index': 23, 'rawText': 'His'}, {'index': 24, 'rawText': 'vlogging'}, {'index': 25, 'rawText': 'channel'}, {'index': 26, 'rawText': ','}, {'index': 27, 

In [21]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm  # Per la barra di progresso

# Lista per salvare i campioni modificati
modified_samples = []

# Iteriamo su tutti i campioni del dataset
for sample in tqdm(ds['train']):
    # Ottieni la colonna SRL
    srl_data = sample['srl']

    # Estrai l'ipotesi
    hypothesis_tokens = srl_data['hypothesis']['tokens']
    hypothesis = ' '.join(token['rawText'] for token in hypothesis_tokens)

    # Estrai le annotazioni dell'ipotesi
    hypothesis_annotations = srl_data['hypothesis']['annotations']

    # Mappiamo i ruoli
    role_mapping = {
        'Agent': 'AGENT',
        'Patient': 'PATIENT'
    }

    # Otteniamo i token dell'ipotesi
    token_map = {token['index']: token['rawText'] for token in hypothesis_tokens}

    # Troviamo AGENT e PATIENT
    agent_text = None
    patient_text = None
    agent_span = None
    patient_span = None

    for annotation in hypothesis_annotations:
        for role in annotation.get('verbatlas', {}).get('roles', []):
            role_name = role['role']
            if role_name in role_mapping:
                span = role['span']
                # Riduci l'ultimo indice dello span di uno
                reduced_span = (max(0, span[0]), max(0, span[1] - 1))
                role_text = ' '.join(token_map[i] for i in range(reduced_span[0], reduced_span[1] + 1))

                if role_mapping[role_name] == 'AGENT':
                    agent_text = role_text
                    agent_span = reduced_span
                elif role_mapping[role_name] == 'PATIENT':
                    patient_text = role_text
                    patient_span = reduced_span

    # Procedi solo se AGENT e PATIENT sono presenti
    if agent_text and patient_text:
        # Crea una lista di token per modificare l'ipotesi
        original_tokens = [token['rawText'] for token in hypothesis_tokens]

        # Creiamo una copia dei token per evitare modifiche non volute durante il processo
        modified_tokens = []
        i = 0

        while i < len(original_tokens):
            # Controlla se i token attuali corrispondono ad AGENT
            if ' '.join(original_tokens[i:i+len(agent_text.split())]) == agent_text:
                # Sostituisci AGENT con PATIENT
                modified_tokens.extend(patient_text.split())
                i += len(agent_text.split())  # Salta i token che compongono AGENT
            # Controlla se i token attuali corrispondono a PATIENT
            elif ' '.join(original_tokens[i:i+len(patient_text.split())]) == patient_text:
                # Sostituisci PATIENT con AGENT
                modified_tokens.extend(agent_text.split())
                i += len(patient_text.split())  # Salta i token che compongono PATIENT
            else:
                # Aggiungi il token originale se non corrisponde né ad AGENT né a PATIENT
                modified_tokens.append(original_tokens[i])
                i += 1

        # Crea la frase modificata
        modified_hypothesis = ' '.join(modified_tokens)

        # Rimuovi eventuali duplicati di spazio (come i punti all'inizio o alla fine)
        modified_hypothesis = ' '.join(modified_hypothesis.split())

        # Gestione della label
        original_label = sample['label']

        if original_label == 'ENTAILMENT':
            new_label = 'CONTRADICTION'
        elif original_label == 'CONTRADICTION':
            new_label = 'ENTAILMENT'
        else:
            new_label = original_label

        # Ora creiamo il dizionario con l'ipotesi modificata e la nuova label
        modified_sample = {
            'id': sample['id'],
            'premise': sample['premise'],  # Lasciamo invariata la premise
            'hypothesis': modified_hypothesis,  # La nostra ipotesi modificata
            'label': new_label,  # La nuova label modificata
            'wsd': sample['wsd'],  # Lasciamo invariato WSD
            'srl': sample['srl']  # Lasciamo invariato SRL
        }

        # Aggiungi il campione modificato alla lista
        modified_samples.append(modified_sample)

# Crea il nuovo dataset modificato con i campioni che hanno avuto AGENT e PATIENT invertiti
modified_dataset = DatasetDict({
    'train': Dataset.from_list(modified_samples)
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset['train'])}")

100%|██████████| 51086/51086 [02:17<00:00, 372.88it/s]


Dimensione del dataset modificato: 2843


In [22]:
# Stampa le ipotesi e le label per i campioni 34 e 46 di ds
print("Dataset originale (ds):")
for idx in [67, 189]:
    sample = ds['train'][idx]
    print(f"Ipotesi {idx}: {sample['hypothesis']}")
    print(f"Label {idx}: {sample['label']}\n")

# Stampa le ipotesi e le label per i primi due campioni del dataset modificato
print("Dataset modificato (modified_dataset):")
for idx in [2, 6]:
    sample = modified_dataset['train'][idx]
    print(f"Ipotesi {idx}: {sample['hypothesis']}")
    print(f"Label {idx}: {sample['label']}\n")


Dataset originale (ds):
Ipotesi 67: Jackie Chan has released an album.
Label 67: ENTAILMENT

Ipotesi 189: Jennifer Hudson has won a Grammy.
Label 189: ENTAILMENT

Dataset modificato (modified_dataset):
Ipotesi 2: an album has released Jackie Chan .
Label 2: CONTRADICTION

Ipotesi 6: a Grammy has won Jennifer Hudson .
Label 6: CONTRADICTION



### Inversione Theme e Attribute

Dimensione del dataset modificato: 19790




In [23]:
import nltk

# Scarica il dataset wordnet
nltk.download('wordnet')

from datasets import load_dataset

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from tqdm import tqdm
from datasets import Dataset, DatasetDict

# Lista per salvare i campioni modificati
modified_samples = []

# Funzione per verificare se una frase ha un senso logico
def has_sensible_roles(theme_text, attribute_text):
    return bool(theme_text) and bool(attribute_text) and len(theme_text.split()) > 1 and len(attribute_text.split()) > 1

# Itera su tutti i campioni del dataset con una barra di progresso
for idx in tqdm(range(len(ds['train'])), desc="Processing samples", unit="sample"):
    sample = ds['train'][idx]

    # Ottieni la colonna SRL
    srl_data = sample['srl']

    # Estrai l'ipotesi
    hypothesis_tokens = srl_data['hypothesis']['tokens']
    hypothesis = ' '.join(token['rawText'] for token in hypothesis_tokens)

    # Estrai le annotazioni dell'ipotesi
    hypothesis_annotations = srl_data['hypothesis']['annotations']

    # Variabili per THEME e ATTRIBUTE
    theme_text = None
    attribute_text = None

    # Troviamo THEME e ATTRIBUTE
    for annotation in hypothesis_annotations:
        for role in annotation.get('verbatlas', {}).get('roles', []):
            if role['role'] == 'Theme':  # THEME
                theme_span = role['span']
                # Riduciamo lo span di 1 (sia l'indice finale che quello iniziale)
                reduced_theme_span = (max(0, theme_span[0]), max(0, theme_span[1] - 1))
                theme_text = ' '.join(token['rawText'] for token in hypothesis_tokens[reduced_theme_span[0]:reduced_theme_span[1] + 1])

            elif role['role'] == 'Attribute':  # ATTRIBUTE
                attribute_span = role['span']
                # Riduciamo lo span di 1 (sia l'indice finale che quello iniziale)
                reduced_attribute_span = (max(0, attribute_span[0]), max(0, attribute_span[1] - 1))
                attribute_text = ' '.join(token['rawText'] for token in hypothesis_tokens[reduced_attribute_span[0]:reduced_attribute_span[1] + 1])

    # Verifica se la frase ha ruoli sensati
    if has_sensible_roles(theme_text, attribute_text):
        # Modifica la frase invertendo THEME e ATTRIBUTE (se entrambi esistono)
        if theme_text and attribute_text:
            # Modifica la frase spostando ATTRIBUTE prima di THEME
            modified_hypothesis = hypothesis.replace(theme_text, 'TEMP_THEME').replace(attribute_text, theme_text).replace('TEMP_THEME', attribute_text)

            # Crea il dizionario con l'ipotesi modificata e la nuova label
            modified_sample = {
                'id': sample['id'],
                'premise': sample['premise'],  # Lasciamo invariata la premise
                'hypothesis': modified_hypothesis,  # La nostra ipotesi modificata
                'label': sample['label'],  # Manteniamo la label originale
                'wsd': sample['wsd'],  # Lasciamo invariato WSD
                'srl': sample['srl']  # Lasciamo invariato SRL
            }

            # Aggiungi il campione modificato alla lista
            modified_samples.append(modified_sample)

# Crea il nuovo dataset modificato
modified_dataset = DatasetDict({
    'train': Dataset.from_list(modified_samples)
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset['train'])}")

Processing samples:  51%|█████     | 26020/51086 [19:15<18:32, 22.53sample/s]    


In [ ]:
# Inserisci gli indici dei campioni che vuoi stampare
original_indices = [0, 1]  # Sostituisci con gli indici desiderati per il dataset originale
modified_indices = [10, 15]    # Sostituisci con gli indici desiderati per il nuovo dataset modificato

# Seleziona i campioni dal dataset originale
original_samples = [ds['train'][idx] for idx in original_indices]

# Seleziona i campioni dal nuovo dataset modificato
modified_samples = [modified_dataset['train'][idx] for idx in modified_indices]

# Stampa i campioni originali
print("Campioni originali:")
for idx, sample in zip(original_indices, original_samples):
    print(f"\nSample index: {idx}")
    print(f"Premise: {sample['premise']}")
    print(f"Ipotesti: {sample['hypothesis']}")
    print(f"Label: {sample['label']}")

# Stampa i campioni modificati
print("\nCampioni modificati:")
for idx, sample in zip(modified_indices, modified_samples):
    print(f"\nSample index: {idx}")
    print(f"Premise: {sample['premise']}")
    print(f"Ipotesti: {sample['hypothesis']}")
    print(f"Label: {sample['label']}")

### Calcolo età rispetto a quando è nato

Dimensione del dataset modificato: 289




In [ ]:
import nltk
from datasets import load_dataset

ds = load_dataset("tommasobonomo/sem_augmented_fever_nli")

In [ ]:
import re
from datasets import load_dataset, Dataset, DatasetDict
from tqdm import tqdm

# Carica il dataset di addestramento
dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli", split='train')

def replace_year_with_relative_time(hypothesis, srl_data):
    for annotation in srl_data.get('hypothesis', {}).get('annotations', []):
        frame_name = annotation.get('verbatlas', {}).get('frameName', '')
        if 'GIVE-BIRTH' in frame_name:
            roles = annotation.get('verbatlas', {}).get('roles', [])
            for role in roles:
                if role['role'] == 'Time':
                    time_span = role.get('span', [])
                    if len(time_span) == 2:
                        start_idx, end_idx = time_span
                        year_tokens = [token['rawText'] for token in srl_data['hypothesis']['tokens'][start_idx:end_idx]]

                        # Rimuovi le parole non numeriche attorno all'anno
                        year_str = ' '.join(year_tokens).replace('in the ', '').replace('the ', '')

                        try:
                            year = int(re.sub(r'\D', '', year_str))  # Estrai solo i numeri
                            if 1900 <= year <= 2100:
                                years_ago = 2024 - year
                                # Trova e sostituisci l'anno nella frase
                                old_hypothesis_pattern = r'\bwas born\b.*?\b\d{4}\b'
                                new_hypothesis = re.sub(old_hypothesis_pattern, f'was born {years_ago} years ago', hypothesis)
                                return new_hypothesis
                        except ValueError:
                            continue
    return hypothesis

# Lista per salvare i campioni modificati
modified_samples = []

# Itera su tutti i campioni del dataset con una barra di progresso
for sample in tqdm(dataset, desc="Processing samples", unit="sample"):
    hypothesis = sample.get('hypothesis', '')
    srl_data = sample.get('srl', {})

    # Sostituisci l'anno con "N years ago"
    new_hypothesis = replace_year_with_relative_time(hypothesis, srl_data)

    # Aggiungi solo se l'ipotesi è stata modificata
    if new_hypothesis != hypothesis:
        modified_sample = {
            'id': sample['id'],
            'premise': sample['premise'],
            'hypothesis': new_hypothesis,
            'label': sample['label'],
            'wsd': sample['wsd'],
            'srl': sample['srl']
        }
        modified_samples.append(modified_sample)

# Crea il nuovo dataset modificato
modified_dataset_born = DatasetDict({
    'train': Dataset.from_list(modified_samples)
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset_born['train'])}")

In [ ]:
print(modified_dataset['train']['hypothesis'][0])
print(modified_dataset['train']['hypothesis'][1])
print(modified_dataset['train']['hypothesis'][2])

### Calcolo età rispetto a quando saremo nel 2050

Dimensione del dataset modificato: 316

In [ ]:
import re
from datasets import load_dataset, Dataset, DatasetDict
from tqdm import tqdm

# Carica il dataset di addestramento
dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli", split='train')

def extract_birth_year(srl_data):
    for annotation in srl_data.get('hypothesis', {}).get('annotations', []):
        frame_name = annotation.get('verbatlas', {}).get('frameName', '')
        if 'GIVE-BIRTH' in frame_name:
            roles = annotation.get('verbatlas', {}).get('roles', [])
            for role in roles:
                if role['role'] == 'Time':
                    time_span = role.get('span', [])
                    if len(time_span) == 2:
                        start_idx, end_idx = time_span
                        year_tokens = [token['rawText'] for token in srl_data['hypothesis']['tokens'][start_idx:end_idx]]

                        # Rimuovi le parole non numeriche attorno all'anno
                        year_str = ' '.join(year_tokens).replace('in the ', '').replace('the ', '')

                        try:
                            year = int(re.sub(r'\D', '', year_str))  # Estrai solo i numeri
                            if 1900 <= year <= 2100:
                                return year
                        except ValueError:
                            continue
    return None

def generate_future_age_hypothesis(hypothesis, birth_year, future_year=2050):
    if birth_year:
        age_in_future = future_year - birth_year
        # Genera la nuova ipotesi basata sul paradigma Age Comparison
        new_hypothesis = f"{hypothesis.replace('was born', f'will be {age_in_future} years old in the {future_year}')}"
        # Rimuovi tutto ciò che segue "in the {future_year}"
        new_hypothesis = re.sub(r'(in the \d{4}).*', r'\1', new_hypothesis)
        return new_hypothesis
    return hypothesis

# Lista per salvare i campioni modificati
modified_samples = []

# Itera su tutti i campioni del dataset con una barra di progresso
for sample in tqdm(dataset, desc="Processing samples", unit="sample"):
    hypothesis = sample.get('hypothesis', '')
    srl_data = sample.get('srl', {})

    # Estrai l'anno di nascita
    birth_year = extract_birth_year(srl_data)

    # Genera la nuova ipotesi
    new_hypothesis = generate_future_age_hypothesis(hypothesis, birth_year)

    # Aggiungi solo se l'ipotesi è stata modificata
    if new_hypothesis != hypothesis:
        modified_sample = {
            'id': sample['id'],
            'premise': sample['premise'],
            'hypothesis': new_hypothesis,
            'label': sample['label'],
            'wsd': sample['wsd'],
            'srl': sample['srl']
        }
        modified_samples.append(modified_sample)

# Crea il nuovo dataset modificato
modified_dataset_next = DatasetDict({
    'train': Dataset.from_list(modified_samples)
})

# Stampa la dimensione del dataset modificato
print(f"Dimensione del dataset modificato: {len(modified_dataset_next['train'])}")

# Mostra i primi esempi per verifica
print("Esempi modificati:")
for sample in modified_samples[:5]:
    print(sample['hypothesis'])

In [ ]:
print(modified_dataset['train']['hypothesis'][0])
print(modified_dataset['train']['hypothesis'][1])
print(modified_dataset['train']['hypothesis'][2])

## Dataset nuovo

In [ ]:
from datasets import concatenate_datasets

# Supponiamo che i dataset siano già definiti
# Concatenazione dei dataset
modified_dataset_train = concatenate_datasets([
    modified_dataset_next['train'],
    modified_dataset_born['train'],
    modified_dataset_hypernyms['train'],
    modified_dataset_antonym['train'],
    modified_dataset_verbs['train'],
    modified_dataset_adjectives['train'],
    modified_dataset_adjectives_2['train']
])

# Crea il DatasetDict finale
modified_dataset_train = DatasetDict({
    'train': modified_dataset_train
})

# Stampa la dimensione del dataset unificato
print(f"Dimensione del dataset unificato: {len(modified_dataset_train['train'])}")